In [3]:
default_installations = True
if default_installations:
    !pip install -q llmlingua llama-index==0.10.7 accelerate datasets llama-index-postprocessor-longllmlingua
else:
    import requests

    text_file_path = "requirements__Ch9_RAGLlamaIndex_Prompt_Compression.txt"
    url = "https://raw.githubusercontent.com/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/main/Chapter9_notebooks/" + text_file_path
    res = requests.get(url)
    with open(text_file_path, "w") as f:
        f.write(res.text)
    !pip install -r requirements__Ch9_RAGLlamaIndex_Prompt_Compression.txt

In [4]:
import torch
import gc
import openai
from datasets import load_dataset
import pandas as pd
from llama_index.core import Document
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.indices.query.schema import QueryBundle
from llama_index.core.response_synthesizers import CompactAndRefine
from llama_index.postprocessor.longllmlingua import LongLLMLinguaPostprocessor
import random
import time
import pickle
import os

In [5]:
gpt_type = "gpt-4o-mini"
num_of_iterations = 60
set_seed = 0
similarity_top_k = 5
target_token = 2 ** 9

In [6]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [7]:
openai.api_key = "..."

In [8]:
dataset = load_dataset("ccdv/arxiv-classification")
dataset_train = dataset["train"]
dataset_train_df = pd.DataFrame(dataset_train)
dataset_train_df_ai = dataset_train_df[dataset_train_df["label"] == 2].reset_index(drop=True)

Generating test split: 100%|██████████| 2500/2500 [00:00<00:00, 15182.08 examples/s]


In [9]:
dataset_train_df_ai.head(1).style.set_properties(**{'text-align': 'left'})

In [10]:
articles = [Document(text=content) for content in dataset_train_df_ai["text"]]

In [11]:
print("'articles'는 리스트이며, 각 요소는 다음과 같은 유형입니다.:", type(articles[0]))

'articles'는 리스트이며, 각 요소는 다음과 같은 유형입니다.: <class 'llama_index.core.schema.Document'>


In [13]:
llm = OpenAI(model=gpt_type)
postprocessor = LongLLMLinguaPostprocessor(
    model_name="NousResearch/Llama-2-7b-hf",
    target_token=target_token,
    rank_method="longllmlingua",
    additional_compress_kwargs={
        "condition_compare": True,
        "condition_in_question": "after",
        "context_budget": "+100",
        "reorder_context": "sort",
        "dynamic_context_compression_ratio": 0.3,
    },
)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
query = "Does this publication involve Reinforcement Learning? Answer in a single word, either Yes or No"
record = {
    "Yes,Yes": 0,
    "No,No": 0,
    "Yes,No": 0,
    "No,Yes": 0,
    "original_tokens": [],
    "compressed_tokens": [],
    "ratios": [],
    "time_nc": [],
    "time_c": [],
}
gc.collect()
torch.cuda.empty_cache()
random.seed(set_seed)
random_iterations = random.sample(range(len(articles)), num_of_iterations)
iteration_counter = 1
for document_index in random_iterations:
    index = VectorStoreIndex.from_documents(articles[document_index:(document_index + 1)])
    print("\n\n                                                 문서 인덱스:", document_index)
    print("                                                 반복 횟수:", iteration_counter, "/", num_of_iterations)
    iteration_counter += 1
    retrieval_start_time = time.time()
    retriever = index.as_retriever(similarity_top_k=similarity_top_k)
    retrieved_context = retriever.retrieve(query)
    context_list = [chunk.get_content() for chunk in retrieved_context]
    retrieval_time = time.time() - retrieval_start_time
    start_time_nc = time.time()
    prompt = "\n\n".join(context_list + [query])
    response_nc = llm.complete(prompt)
    time_nc = time.time() - start_time_nc
    print("------------------------------------------------ 비압축 방식 응답: " + str(response_nc))
    original_contexts = "\n\n".join(context_list)
    original_tokens = postprocessor._llm_lingua.get_token_length(original_contexts)
    del prompt
    del context_list
    del original_contexts
    gc.collect()
    torch.cuda.empty_cache()
    start_time_c = time.time()
    new_retrieved_context = postprocessor.postprocess_nodes(
        retrieved_context,
        query_bundle=QueryBundle(query_str=query)
    )
    response_c = CompactAndRefine().synthesize(query, new_retrieved_context)
    time_c = time.time() - start_time_c
    print("------------------------------------------------ 압축 방식 응답: " + str(response_c))
    compressed_contexts = "\n\n".join([chunk.get_content() for chunk in new_retrieved_context])
    compressed_tokens = postprocessor._llm_lingua.get_token_length(compressed_contexts)
    ratio = original_tokens / (compressed_tokens + 1)
    print("원래 토큰 수:", original_tokens)
    print("압축된 토큰 수:", compressed_tokens)
    print("압축 비율:", f"{ratio:.2f}배")
    record[str(response_nc).replace(".", "") + "," + str(response_c).replace(".", "")] += 1
    record["original_tokens"].append(original_tokens)
    record["compressed_tokens"].append(compressed_tokens)
    record["ratios"].append(ratio)
    record["time_nc"].append(time_nc + retrieval_time)
    record["time_c"].append(time_c + retrieval_time)
    del compressed_contexts
    del new_retrieved_context
    del retrieved_context
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
num_of_iterations_in_practice = len(record["original_tokens"])
counts_of_agreements = record["Yes,Yes"] + record["No,No"]
counts_of_disagreements = record["Yes,No"] + record["No,Yes"]
reduction_in_tokens = round(100 * (1 - sum(record["compressed_tokens"]) / sum(record["original_tokens"])))

In [ ]:
print("일치 횟수:", str(counts_of_agreements), "전체 사례", str(num_of_iterations_in_practice), "중")
print("불일치 횟수:", str(counts_of_disagreements), "전체 사례", str(num_of_iterations_in_practice), "중")
print(f"일치율: {round(100 * counts_of_agreements / num_of_iterations_in_practice)}%")

In [ ]:
print(f"비압축 방식: {num_of_iterations_in_practice}번 호출에서 전송된 총 토큰 수:", str(sum(record["original_tokens"])))
print(f"압축 방식:   {num_of_iterations_in_practice}번 호출에서 전송된 총 토큰 수:", str(sum(record["compressed_tokens"])))
print(f"토큰 감소율: {reduction_in_tokens}%")
print("압축 비율:", f"{1 / (1 - reduction_in_tokens / 100):.2f}배")

In [ ]:
print(f"비압축 방식: {num_of_iterations_in_practice}번 호출 동안의 총 반복 시간:", str(round(sum(record["time_nc"]))))
print(f"압축 방식:   {num_of_iterations_in_practice}번 호출 동안의 총 반복 시간:", str(round(sum(record["time_c"]))))

In [ ]:
reduction_in_agreement = round(100 * (record["Yes,No"] + record["No,Yes"]) / num_of_iterations_in_practice)
print(f"분류 일치율이 {reduction_in_agreement}% 감소하는 동안, 토큰 감소로 인한 비용 절감율은 {reduction_in_tokens}%입니다.\n")
print(f"즉, 압축 전 $100의 비용이 압축 후에는 ${100 - reduction_in_tokens}로 줄어듭니다.")
print("*두 접근 방식 간의 불일치는 어느 한쪽이 올바른 결과일 수 있음을 의미하며, 압축되지 않은 방식이 더 신뢰할 수 있다고 가정해서는 안 됩니다. 압축되지 않은 방식 또한 희소할 수 있기 때문입니다.")
print("**압축기 사용에는 GPU가 필요한 로컬 LLM 활용과 같은 다른 절충이 따른다는 점을 기억하세요.")

In [ ]:
out = open("record.pickle", "wb")
pickle.dump(record, out)
out.close()